In [1]:
%%writefile streamlit1_app.py
import streamlit as st
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout

st.title("📈 Stock Price Prediction with LSTM")

uploaded_file = st.file_uploader("Upload a CSV file with Date and Close columns", type=["csv"])
if uploaded_file:
    df = pd.read_csv(uploaded_file)

    # Data Cleaning
    df['Date'] = pd.to_datetime(df['Date'], errors='coerce')
    df.dropna(subset=['Date', 'Close'], inplace=True)
    df.sort_values('Date', inplace=True)
    df.reset_index(drop=True, inplace=True)

    # Visualization
    st.subheader("📊 Historical Close Price")
    st.line_chart(df.set_index('Date')['Close'])

    # LSTM Preparation
    data = df[['Close']].values
    scaler = MinMaxScaler()
    scaled_data = scaler.fit_transform(data)

    seq_len = 60
    X, y = [], []
    for i in range(seq_len, len(scaled_data)):
        X.append(scaled_data[i - seq_len:i, 0])
        y.append(scaled_data[i, 0])
    X, y = np.array(X), np.array(y)
    X = np.reshape(X, (X.shape[0], X.shape[1], 1))

    # Train/test split
    split = int(0.8 * len(X))
    X_train, X_test = X[:split], X[split:]
    y_train, y_test = y[:split], y[split:]

    # Build LSTM Model
    model = Sequential()
    model.add(LSTM(50, return_sequences=True, input_shape=(X.shape[1], 1)))
    model.add(Dropout(0.2))
    model.add(LSTM(50, return_sequences=False))
    model.add(Dropout(0.2))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mean_squared_error')
    model.fit(X_train, y_train, epochs=5, batch_size=32, verbose=0)

    # Prediction
    predictions = model.predict(X_test)
    predictions = scaler.inverse_transform(predictions.reshape(-1, 1))
    y_actual = scaler.inverse_transform(y_test.reshape(-1, 1))
    # --- Future Prediction ---
    st.subheader("📅 Predict Future Stock Prices")

    n_days = st.slider("Select number of days to forecast into the future", min_value=1, max_value=30, value=7)

    # Start from the last 60 values
    last_seq = scaled_data[-seq_len:].reshape(1, seq_len, 1)
    future_predictions = []

for _ in range(n_days):
   next_pred = model.predict(last_seq)[0][0]
   future_predictions.append(next_pred)
    # Slide the window
   last_seq = np.append(last_seq[:, 1:, :], [[[next_pred]]], axis=1)

    # Inverse transform to original price scale
future_predictions = scaler.inverse_transform(np.array(future_predictions).reshape(-1, 1))

    # Generate future dates
last_date = df['Date'].iloc[-1]
future_dates = pd.date_range(last_date + pd.Timedelta(days=1), periods=n_days)

    # Create dataframe for predictions
future_df = pd.DataFrame({'Date': future_dates, 'Predicted_Close': future_predictions.flatten()})
st.dataframe(future_df)

    # Plot future values
st.subheader("📈 Future Forecast Plot")
fig2, ax2 = plt.subplots(figsize=(12, 5))
ax2.plot(df['Date'], df['Close'], label="Historical")
ax2.plot(future_df['Date'], future_df['Predicted_Close'], label="Forecast", linestyle='dashed')
ax2.set_xlabel("Date")
ax2.set_ylabel("Price")
ax2.legend()
st.pyplot(fig2)


    # Plotting
st.subheader("🔮 Predicted vs Actual Close Price")
fig, ax = plt.subplots(figsize=(12, 5))
ax.plot(y_actual, label="Actual")
ax.plot(predictions, label="Predicted")
ax.set_xlabel("Time")
ax.set_ylabel("Price")
ax.legend()
st.pyplot(fig)


Overwriting streamlit1_app.py


In [ ]:
!streamlit run streamlit1_app.py


  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8503
  Network URL: http://192.168.29.221:8503

2025-06-09 22:43:02.396720: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-06-09 22:43:02.404362: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-06-09 22:43:02.434797: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1749489182.485780   26002 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1749489182.500616   26002 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 